# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
weather_data=pd.read_csv(r'/Users/doggger/Desktop/personal-data/api challenge/python-api-challenge/output_data\weather.csv')
weather_data.head()

,city,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,shakawe,-18.37,21.85,96.21,14,73,3.44,1603901952,BW
1,cape town,-33.93,18.42,62.80,67,20,19.46,1603901700,ZA
2,vytegra,61.01,36.45,45.52,84,30,8.10,1603901947,RU
3,husavik,66.04,-17.34,41.00,75,75,16.11,1603901953,IS
4,brewster,41.40,-73.62,47.95,100,90,4.70,1603901787,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
#configure gmaps
gmaps.configure(api_key=g_key)

#create df for locations and weight to pass into gmaps
locations = weather_data[["lat", "lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

#add heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)

#display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
hotel_df=weather_data.loc[(weather_data['Max Temp'] > 60) &
                          (weather_data['Max Temp'] > 85) &
                          (weather_data['Humidity'] < 60) &
                          (weather_data['Wind Speed'] < 10) &
                          (weather_data['Cloudiness'] < 25)]
hotel_df.dropna(axis=0)
hotel_df.head()

,city,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
37,bilma,18.69,12.92,86.18,13,0,6.06,1603901956,NE
71,ayorou,14.73,0.92,97.54,8,0,7.16,1603902018,NE
121,puerto quijarro,-17.78,-57.77,101.25,32,4,7.18,1603902083,BO
129,aswan,24.09,32.91,87.64,16,0,3.80,1603902084,EG
134,doctor juan manuel frutos,-25.37,-55.80,92.05,31,0,2.46,1603902084,PY


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
#Makes call to google API for the first row data. NOTE: for code checking only
hotel_df['Hotel Name'] = ""
target_radius = 5000
target_type = "lodging"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_coordinates= '18.69, 12.92'

params = {
"location": target_coordinates,
"radius": target_radius,
"type": target_type,
"key": g_key}

response = requests.get(base_url, params=params).json()
print(json.dumps(response, indent=4, sort_keys=True))
print(response["results"][0]["name"])

    

C:\Users\Doggger\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 18.6929789,
                    "lng": 12.9176518
                },
                "viewport": {
                    "northeast": {
                        "lat": 18.6943387802915,
                        "lng": 12.9190450802915
                    },
                    "southwest": {
                        "lat": 18.6916408197085,
                        "lng": 12.9163471197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Expeditions T\u00e9n\u00e9re voyages",
            "place_id": "ChIJK367YL5dcBERK29v6zybUaw",
            "plus_code": {
                "compound_code": "MWV9+53 Bilma, Niger",
                "global_code": "7FCJMWV9+53"
            },
            "ra

In [45]:
hotel_df['Hotel Name'] = ""
target_radius = 5000
target_type = "lodging"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lt=row['lat']
    lg=row['lng']
    cty=row['city']
    target_coordinates = f'{lt},{lg}' 

# set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}

#run a request using our params dictionary for each row in hotel_df
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
    except:
        print(f'No hotel found for {cty}')
    

C:\Users\Doggger\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Doggger\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


No hotel found for siguiri
No hotel found for karia
No hotel found for mayahi
No hotel found for kande


In [46]:
hotel_df.head()

,city,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
37,bilma,18.69,12.92,86.18,13,0,6.06,1603901956,NE,Expeditions Ténére voyages
71,ayorou,14.73,0.92,97.54,8,0,7.16,1603902018,NE,Hotel Amenokal
121,puerto quijarro,-17.78,-57.77,101.25,32,4,7.18,1603902083,BO,Hotel palance
129,aswan,24.09,32.91,87.64,16,0,3.80,1603902084,EG,Nuba Nile
134,doctor juan manuel frutos,-25.37,-55.80,92.05,31,0,2.46,1603902084,PY,Balneario Las Burbujas


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [54]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))